In [111]:
# ! pip install medpy pydicom

* https://www.kaggle.com/saurava/braintumor-eda - useful notebook for BRATS 2015 dataset for unstanding dataset structure and visualization

* VSD.Brain_3more.XX.XX.OT - is directory with ground truth masks

* We will use T1_c images - the same as were used in the initial dataset

## Train-Test csv Creation

In [101]:
from medpy.io import load
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.utils import shuffle

In [60]:
df = pd.DataFrame({'path':[], 'label_path':[]})

In [98]:
def get_label_path(path_to_person):
    for f in os.listdir(path_to_person):
        if 'more' in f :
            for i in os.listdir(f'{path_to_person}/{f}'):
                if '.mha' in i:
                    return f'{path_to_person}/{f}/{i}'

In [133]:
path_to_image_data = '/Users/polinaturiseva/Desktop/cv_inno/project/brats/BRATS-2/Image_Data'

path = []
label_path = []
index = []
view = []

for fold in os.listdir(path_to_image_data):
    for person in os.listdir(f'{path_to_image_data}/{fold}'):
        after_person = os.listdir(f'{path_to_image_data}/{fold}/{person}')
        for f in after_person:
            if f=='VSD.Brain.XX.O.MR_T1c':
                for i in os.listdir(f'{path_to_image_data}/{fold}/{person}/{f}'):
                    if '.mha' in i:
                        label = get_label_path(f'{path_to_image_data}/{fold}/{person}')
                        mage_data, _ = load(f'{path_to_image_data}/{fold}/{person}/{f}/{i}')
                        
                        for j in range(mage_data.shape[0]):
                            if sum(sum(mage_data[j, :, :]))!=0:
                                view.append(0)
                                index.append(j)
                                path.append(f'{path_to_image_data}/{fold}/{person}/{f}/{i}')
                                label_path.append(label)
                                
                        for j in range(mage_data.shape[1]):
                            if sum(sum(mage_data[:, j, :]))!=0:
                                view.append(1)
                                index.append(j)
                                path.append(f'{path_to_image_data}/{fold}/{person}/{f}/{i}')
                                label_path.append(label)
                                
                        for j in range(mage_data.shape[2]):
                            if sum(sum(mage_data[:, :, j]))!=0:
                                view.append(2)
                                index.append(j)
                                path.append(f'{path_to_image_data}/{fold}/{person}/{f}/{i}')
                                label_path.append(label)

 
df = pd.DataFrame({'path':path, 'label_path':label_path, 'index': index, 'view':view })
df.to_csv('all_data.csv')
df = shuffle(df)
df[:int(len(df)*0.2)].to_csv('data_test.csv')
df[int(len(df)*0.2):].to_csv('data_train.csv')

## Check for negative cases

In [93]:
count = 0
count_nonzero = 0 
for i in path:
    mage_data, image_header = load(i)
    count += mage_data.shape[0]
    for j in range(mage_data.shape[0]):
        if sum(sum(mage_data[j, :, :]))!=0:
            count_nonzero += 1
            
print(count)
print(count_nonzero)

5417
4199


In [116]:
count = 0
count_nonzero = 0 
non_zero_matrixes = []
for i in set(label_path):
    mage_data, image_header = load(i)
    count += mage_data.shape[0]
    for j in range(mage_data.shape[0]):
        if sum(sum(mage_data[j, :, :]))!=0:
            count_nonzero += 1
            non_zero_matrixes.append(mage_data[j, :, :])
            
print(count)
print(count_nonzero)

5417
1789
